# Credit Risk Resampling Techniques

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter

# Read the CSV and Perform Basic Data Cleaning

In [3]:
columns = [
    "loan_amnt", "int_rate", "installment", "home_ownership",
    "annual_inc", "verification_status", "issue_d", "loan_status",
    "pymnt_plan", "dti", "delinq_2yrs", "inq_last_6mths",
    "open_acc", "pub_rec", "revol_bal", "total_acc",
    "initial_list_status", "out_prncp", "out_prncp_inv", "total_pymnt",
    "total_pymnt_inv", "total_rec_prncp", "total_rec_int", "total_rec_late_fee",
    "recoveries", "collection_recovery_fee", "last_pymnt_amnt", "next_pymnt_d",
    "collections_12_mths_ex_med", "policy_code", "application_type", "acc_now_delinq",
    "tot_coll_amt", "tot_cur_bal", "open_acc_6m", "open_act_il",
    "open_il_12m", "open_il_24m", "mths_since_rcnt_il", "total_bal_il",
    "il_util", "open_rv_12m", "open_rv_24m", "max_bal_bc",
    "all_util", "total_rev_hi_lim", "inq_fi", "total_cu_tl",
    "inq_last_12m", "acc_open_past_24mths", "avg_cur_bal", "bc_open_to_buy",
    "bc_util", "chargeoff_within_12_mths", "delinq_amnt", "mo_sin_old_il_acct",
    "mo_sin_old_rev_tl_op", "mo_sin_rcnt_rev_tl_op", "mo_sin_rcnt_tl", "mort_acc",
    "mths_since_recent_bc", "mths_since_recent_inq", "num_accts_ever_120_pd", "num_actv_bc_tl",
    "num_actv_rev_tl", "num_bc_sats", "num_bc_tl", "num_il_tl",
    "num_op_rev_tl", "num_rev_accts", "num_rev_tl_bal_gt_0",
    "num_sats", "num_tl_120dpd_2m", "num_tl_30dpd", "num_tl_90g_dpd_24m",
    "num_tl_op_past_12m", "pct_tl_nvr_dlq", "percent_bc_gt_75", "pub_rec_bankruptcies",
    "tax_liens", "tot_hi_cred_lim", "total_bal_ex_mort", "total_bc_limit",
    "total_il_high_credit_limit", "hardship_flag", "debt_settlement_flag"
]

target = ["loan_status"]

In [4]:
# Load the data
file_path = Path('Resources/LoanStats_2019Q1.csv')
df = pd.read_csv(file_path, skiprows=1)[:-2]
df = df.loc[:, columns].copy()

# Drop the null columns where all values are null
df = df.dropna(axis='columns', how='all')

# Drop the null rows
df = df.dropna()

# Remove the `Issued` loan status
issued_mask = df['loan_status'] != 'Issued'
df = df.loc[issued_mask]

# convert interest rate to numerical
df['int_rate'] = df['int_rate'].str.replace('%', '')
df['int_rate'] = df['int_rate'].astype('float') / 100


# Convert the target column values to low_risk and high_risk based on their values
x = {'Current': 'low_risk'}   
df = df.replace(x)

x = dict.fromkeys(['Late (31-120 days)', 'Late (16-30 days)', 'Default', 'In Grace Period'], 'high_risk')    
df = df.replace(x)

df.reset_index(inplace=True, drop=True)

df.head()

,loan_amnt,int_rate,installment,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,dti,...,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,debt_settlement_flag
0,10500.0,0.1719,375.35,RENT,66000.0,Source Verified,Mar-2019,low_risk,n,27.24,...,85.7,100.0,0.0,0.0,65687.0,38199.0,2000.0,61987.0,N,N
1,25000.0,0.2000,929.09,MORTGAGE,105000.0,Verified,Mar-2019,low_risk,n,20.23,...,91.2,50.0,1.0,0.0,271427.0,60641.0,41200.0,49197.0,N,N
2,20000.0,0.2000,529.88,MORTGAGE,56000.0,Verified,Mar-2019,low_risk,n,24.26,...,66.7,50.0,0.0,0.0,60644.0,45684.0,7500.0,43144.0,N,N
3,10000.0,0.1640,353.55,RENT,92000.0,Verified,Mar-2019,low_risk,n,31.44,...,100.0,50.0,1.0,0.0,99506.0,68784.0,19700.0,76506.0,N,N
4,22000.0,0.1474,520.39,MORTGAGE,52000.0,Not Verified,Mar-2019,low_risk,n,18.76,...,100.0,0.0,0.0,0.0,219750.0,25919.0,27600.0,20000.0,N,N


# Split the Data into Training and Testing

In [5]:
# Check data types
df.dtypes

loan_amnt                     float64
int_rate                      float64
installment                   float64
home_ownership                 object
annual_inc                    float64
                               ...   
total_bal_ex_mort             float64
total_bc_limit                float64
total_il_high_credit_limit    float64
hardship_flag                  object
debt_settlement_flag           object
Length: 86, dtype: object

In [6]:
# Find strings in df
[ col  for col, dt in df.dtypes.items() if dt == object]

['home_ownership',
 'verification_status',
 'issue_d',
 'loan_status',
 'pymnt_plan',
 'initial_list_status',
 'next_pymnt_d',
 'application_type',
 'hardship_flag',
 'debt_settlement_flag']

In [7]:
# Convert string to numeric values for loan_status
loan_status_dict = {'low_risk': 0, 'high_risk': 1}
df2 = df.replace({'loan_status': loan_status_dict})
#find strings in df
[ col  for col, dt in df2.dtypes.items() if dt == object]

['home_ownership',
 'verification_status',
 'issue_d',
 'pymnt_plan',
 'initial_list_status',
 'next_pymnt_d',
 'application_type',
 'hardship_flag',
 'debt_settlement_flag']

In [8]:
# Convert rest of dataframe from string to numberic values
df2=pd.get_dummies(df2)
df2.head()

,loan_amnt,int_rate,installment,annual_inc,loan_status,dti,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,...,issue_d_Mar-2019,pymnt_plan_n,initial_list_status_f,initial_list_status_w,next_pymnt_d_Apr-2019,next_pymnt_d_May-2019,application_type_Individual,application_type_Joint App,hardship_flag_N,debt_settlement_flag_N
0,10500.0,0.1719,375.35,66000.0,0,27.24,0.0,0.0,8.0,0.0,...,1,1,0,1,0,1,1,0,1,1
1,25000.0,0.2000,929.09,105000.0,0,20.23,0.0,0.0,17.0,1.0,...,1,1,0,1,0,1,1,0,1,1
2,20000.0,0.2000,529.88,56000.0,0,24.26,0.0,0.0,8.0,0.0,...,1,1,0,1,0,1,1,0,1,1
3,10000.0,0.1640,353.55,92000.0,0,31.44,0.0,1.0,10.0,1.0,...,1,1,0,1,0,1,1,0,1,1
4,22000.0,0.1474,520.39,52000.0,0,18.76,0.0,1.0,14.0,0.0,...,1,1,0,1,0,1,1,0,1,1


In [9]:
# Find strings in df2
[ col  for col, dt in df2.dtypes.items() if dt == object]

[]

In [10]:
# Create our features
X = df2.drop("loan_status", axis=1)

# Create the training variables
# converting string values into numerical ones using get_dummies()
X_dummies = pd.get_dummies(X)
X_dummies

,loan_amnt,int_rate,installment,annual_inc,dti,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,revol_bal,...,issue_d_Mar-2019,pymnt_plan_n,initial_list_status_f,initial_list_status_w,next_pymnt_d_Apr-2019,next_pymnt_d_May-2019,application_type_Individual,application_type_Joint App,hardship_flag_N,debt_settlement_flag_N
0,10500.0,0.1719,375.35,66000.0,27.24,0.0,0.0,8.0,0.0,1609.0,...,1,1,0,1,0,1,1,0,1,1
1,25000.0,0.2000,929.09,105000.0,20.23,0.0,0.0,17.0,1.0,18368.0,...,1,1,0,1,0,1,1,0,1,1
2,20000.0,0.2000,529.88,56000.0,24.26,0.0,0.0,8.0,0.0,13247.0,...,1,1,0,1,0,1,1,0,1,1
3,10000.0,0.1640,353.55,92000.0,31.44,0.0,1.0,10.0,1.0,17996.0,...,1,1,0,1,0,1,1,0,1,1
4,22000.0,0.1474,520.39,52000.0,18.76,0.0,1.0,14.0,0.0,9091.0,...,1,1,0,1,0,1,1,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
68812,10000.0,0.1502,346.76,26000.0,9.60,0.0,0.0,9.0,0.0,2684.0,...,0,1,0,1,0,1,1,0,1,1
68813,12000.0,0.2727,368.37,63000.0,29.07,0.0,0.0,8.0,0.0,13314.0,...,0,1,0,1,0,1,1,0,1,1
68814,5000.0,0.1992,185.62,52000.0,14.86,0.0,0.0,5.0,1.0,3715.0,...,0,1,0,1,0,1,1,0,1,1
68815,40000.0,0.0646,1225.24,520000.0,9.96,0.0,1.0,21.0,0.0,59529.0,...,0,1,1,0,0,1,1,0,1,1


In [11]:
# Create our target
y = df2['loan_status']
y.unique()

array([0, 1], dtype=int64)

In [12]:
# Check the balance of our target values
y.value_counts()

0    68470
1      347
Name: loan_status, dtype: int64

In [13]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)
Counter(y_train)

Counter({0: 51366, 1: 246})

# Oversampling

In this section, you will compare two oversampling algorithms to determine which algorithm results in the best performance. You will oversample the data using the naive random oversampling algorithm and the SMOTE algorithm. For each algorithm, be sure to complete the folliowing steps:

1. View the count of the target classes using `Counter` from the collections library. 
3. Use the resampled data to train a logistic regression model.
3. Calculate the balanced accuracy score from sklearn.metrics.
4. Print the confusion matrix from sklearn.metrics.
5. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.

Note: Use a random state of 1 for each sampling algorithm to ensure consistency between tests

### Naive Random Oversampling

In [14]:
# Resample the training data with the RandomOversampler
from imblearn.over_sampling import RandomOverSampler
ros = RandomOverSampler(random_state=1)
X_resampled, y_resampled = ros.fit_resample(X_train, y_train)

Counter(y_resampled)

Counter({0: 51366, 1: 51366})

In [15]:
# Train the Logistic Regression model using the resampled data
from sklearn.linear_model import LogisticRegression

model = LogisticRegression(solver='lbfgs', random_state=1)
model.fit(X_resampled, y_resampled)

LogisticRegression(random_state=1)

In [16]:
# Display the confusion matrix
from sklearn.metrics import confusion_matrix

y_pred = model.predict(X_test)

cm = confusion_matrix(y_test, y_pred)
cm_df = pd.DataFrame(
    cm, index=["Actual 0 (Low Risk)", "Actual 1 (High Risk)"], columns=["Predicted 0 (Low Risk)", "Predicted 1 (High Risk)"]
)
cm_df

,Predicted 0 (Low Risk),Predicted 1 (High Risk)
Actual 0 (Low Risk),10292,6812
Actual 1 (High Risk),29,72


In [17]:
# Calculated the balanced accuracy score
from sklearn.metrics import balanced_accuracy_score
balanced_accuracy_score(y_test, y_pred)

0.6573009382322703

In [18]:
# Print the imbalanced classification report
from imblearn.metrics import classification_report_imbalanced
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

          0       1.00      0.60      0.71      0.75      0.65      0.42     17104
          1       0.01      0.71      0.60      0.02      0.65      0.43       101

avg / total       0.99      0.60      0.71      0.75      0.65      0.42     17205



In [19]:
# Displaying results
print("Random Oversampling")
display(cm_df)
print(f"Accuracy Score : {balanced_accuracy_score(y_test, y_pred)}")
print("Classification Report")
print(classification_report_imbalanced(y_test, y_pred))

Random Oversampling


,Predicted 0 (Low Risk),Predicted 1 (High Risk)
Actual 0 (Low Risk),10292,6812
Actual 1 (High Risk),29,72


Accuracy Score : 0.6573009382322703
Classification Report
                   pre       rec       spe        f1       geo       iba       sup

          0       1.00      0.60      0.71      0.75      0.65      0.42     17104
          1       0.01      0.71      0.60      0.02      0.65      0.43       101

avg / total       0.99      0.60      0.71      0.75      0.65      0.42     17205



### SMOTE Oversampling

In [20]:
# Resample the training data with SMOTE
from imblearn.over_sampling import SMOTE
X_resampled, y_resampled = SMOTE(random_state=1, sampling_strategy='auto').fit_resample(
    X_train, y_train
)
Counter(y_resampled)

Counter({0: 51366, 1: 51366})

In [21]:
# Train the Logistic Regression model using the resampled data
model = LogisticRegression(solver='lbfgs', random_state=1)
model.fit(X_resampled, y_resampled)

LogisticRegression(random_state=1)

In [22]:
# Display the confusion matrix
y_pred = model.predict(X_test)

cm = confusion_matrix(y_test, y_pred)
cm_df2 = pd.DataFrame(
    cm, index=["Actual 0 (Low Risk)", "Actual 1 (High Risk)"], columns=["Predicted 0 (Low Risk)", "Predicted 1 (High Risk)"]
)
cm_df2

,Predicted 0 (Low Risk),Predicted 1 (High Risk)
Actual 0 (Low Risk),11816,5288
Actual 1 (High Risk),37,64


In [23]:
# Calculated the balanced accuracy score
balanced_accuracy_score(y_test, y_pred)

0.6622479600626106

In [24]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

          0       1.00      0.69      0.63      0.82      0.66      0.44     17104
          1       0.01      0.63      0.69      0.02      0.66      0.44       101

avg / total       0.99      0.69      0.63      0.81      0.66      0.44     17205



In [25]:
# Displaying results
print("SMOTE Oversampling")
display(cm_df2)
print(f"Accuracy Score : {balanced_accuracy_score(y_test, y_pred)}")
print("Classification Report")
print(classification_report_imbalanced(y_test, y_pred))

SMOTE Oversampling


,Predicted 0 (Low Risk),Predicted 1 (High Risk)
Actual 0 (Low Risk),11816,5288
Actual 1 (High Risk),37,64


Accuracy Score : 0.6622479600626106
Classification Report
                   pre       rec       spe        f1       geo       iba       sup

          0       1.00      0.69      0.63      0.82      0.66      0.44     17104
          1       0.01      0.63      0.69      0.02      0.66      0.44       101

avg / total       0.99      0.69      0.63      0.81      0.66      0.44     17205



# Undersampling

In this section, you will test an undersampling algorithms to determine which algorithm results in the best performance compared to the oversampling algorithms above. You will undersample the data using the Cluster Centroids algorithm and complete the folliowing steps:

1. View the count of the target classes using `Counter` from the collections library. 
3. Use the resampled data to train a logistic regression model.
3. Calculate the balanced accuracy score from sklearn.metrics.
4. Print the confusion matrix from sklearn.metrics.
5. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.

Note: Use a random state of 1 for each sampling algorithm to ensure consistency between tests

In [26]:
# Resample the data using the ClusterCentroids resampler
# Warning: This is a large dataset, and this step may take some time to complete
from imblearn.under_sampling import RandomUnderSampler
ros = RandomUnderSampler(random_state=1)
X_resampled, y_resampled = ros.fit_resample(X_train, y_train)
Counter(y_resampled)

Counter({0: 246, 1: 246})

In [27]:
# Train the Logistic Regression model using the resampled data
model = LogisticRegression(solver='lbfgs', random_state=1)
model.fit(X_resampled, y_resampled)

LogisticRegression(random_state=1)

In [28]:
# Display the confusion matrix
y_pred = model.predict(X_test)

cm = confusion_matrix(y_test, y_pred)
cm_df3 = pd.DataFrame(
    cm, index=["Actual 0 (Low Risk)", "Actual 1 (High Risk)"], columns=["Predicted 0 (Low Risk)", "Predicted 1 (High Risk)"]
)
cm_df3

,Predicted 0 (Low Risk),Predicted 1 (High Risk)
Actual 0 (Low Risk),9185,7919
Actual 1 (High Risk),42,59


In [29]:
# Calculated the balanced accuracy score
balanced_accuracy_score(y_test, y_pred)

0.5605836513258435

In [30]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

          0       1.00      0.54      0.58      0.70      0.56      0.31     17104
          1       0.01      0.58      0.54      0.01      0.56      0.32       101

avg / total       0.99      0.54      0.58      0.69      0.56      0.31     17205



In [31]:
# Displaying results
print("Undersampling")
display(cm_df3)
print(f"Accuracy Score : {balanced_accuracy_score(y_test, y_pred)}")
print("Classification Report")
print(classification_report_imbalanced(y_test, y_pred))

Undersampling


,Predicted 0 (Low Risk),Predicted 1 (High Risk)
Actual 0 (Low Risk),9185,7919
Actual 1 (High Risk),42,59


Accuracy Score : 0.5605836513258435
Classification Report
                   pre       rec       spe        f1       geo       iba       sup

          0       1.00      0.54      0.58      0.70      0.56      0.31     17104
          1       0.01      0.58      0.54      0.01      0.56      0.32       101

avg / total       0.99      0.54      0.58      0.69      0.56      0.31     17205



# Combination (Over and Under) Sampling

In this section, you will test a combination over- and under-sampling algorithm to determine if the algorithm results in the best performance compared to the other sampling algorithms above. You will resample the data using the SMOTEENN algorithm and complete the folliowing steps:

1. View the count of the target classes using `Counter` from the collections library. 
3. Use the resampled data to train a logistic regression model.
3. Calculate the balanced accuracy score from sklearn.metrics.
4. Print the confusion matrix from sklearn.metrics.
5. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.

Note: Use a random state of 1 for each sampling algorithm to ensure consistency between tests

In [32]:
# Resample the training data with SMOTEENN
# Warning: This is a large dataset, and this step may take some time to complete
from imblearn.combine import SMOTEENN

smote_enn = SMOTEENN(random_state=1)
X_resampled, y_resampled = smote_enn.fit_resample(X, y)
Counter(y_resampled)

Counter({0: 62022, 1: 68458})

In [33]:
# Train the Logistic Regression model using the resampled data
model = LogisticRegression(solver='lbfgs', random_state=1)
model.fit(X_resampled, y_resampled)

LogisticRegression(random_state=1)

In [34]:
# Display the confusion matrix
y_pred = model.predict(X_test)

cm = confusion_matrix(y_test, y_pred)
cm_df4 = pd.DataFrame(
    cm, index=["Actual 0 (Low Risk)", "Actual 1 (High Risk)"], columns=["Predicted 0 (Low Risk)", "Predicted 1 (High Risk)"]
)
cm_df4

,Predicted 0 (Low Risk),Predicted 1 (High Risk)
Actual 0 (Low Risk),10301,6803
Actual 1 (High Risk),25,76


In [35]:
# Calculated the balanced accuracy score
balanced_accuracy_score(y_test, y_pred)

0.6773660147820207

In [36]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

          0       1.00      0.60      0.75      0.75      0.67      0.45     17104
          1       0.01      0.75      0.60      0.02      0.67      0.46       101

avg / total       0.99      0.60      0.75      0.75      0.67      0.45     17205



In [37]:
# Displaying results
print("SMOTEEN Combined Over and Undersampling")
display(cm_df4)
print(f"Accuracy Score : {balanced_accuracy_score(y_test, y_pred)}")
print("Classification Report")
print(classification_report_imbalanced(y_test, y_pred))

SMOTEEN Combined Over and Undersampling


,Predicted 0 (Low Risk),Predicted 1 (High Risk)
Actual 0 (Low Risk),10301,6803
Actual 1 (High Risk),25,76


Accuracy Score : 0.6773660147820207
Classification Report
                   pre       rec       spe        f1       geo       iba       sup

          0       1.00      0.60      0.75      0.75      0.67      0.45     17104
          1       0.01      0.75      0.60      0.02      0.67      0.46       101

avg / total       0.99      0.60      0.75      0.75      0.67      0.45     17205

